In [ ]:
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Initial reference behavioral vector (D_ref) and user context
D_ref = np.array([79, 330, 9, 278, 74, 5, 34.32, 13, 154, 23], dtype=np.float32)
age = 65
idle_threshold = 3
idle_count = 0
travel_flag = 0

# Locations
last_login_loc = (28.7041, 77.1025) #pevious session waali
current_loc    = (28.7041, 77.1025)
#ye woh 30 sec waali location update
prev_loc       = (28.5355, 77.3910)
latest_loc     = (28.5355, 77.3910)

# New 30s feature vector
v = np.array([79, 320, 9, 278, 74, 5, 34.32, 13, 155, 23], dtype=np.float32)

# EMA (exponential moving averages ) parameters
alpha_mean = 0.05
alpha_std = 0.02
distance_update_threshold = 1.5  # taaki outliers use na ho update ke liye


def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat, dlon = radians(lat2-lat1), radians(lon2-lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * atan2(sqrt(a), sqrt(1 - a)) * R

def estimate_std(D):
    return np.maximum(1e-2, np.abs(D * 0.10))  # 10% static tolerance

def compute_anomaly_score(v, D, D_std, age):
    if np.count_nonzero(v) < 4:
        return None, None
    z = np.abs((v - D) / (D_std + 1e-8))
    base_score = np.mean(z)
    if age >= 60:
        base_score *= 1.15
    return base_score, z

def rule_based_checks(v, last_loc, curr_loc, prev_loc, latest_loc):
    flags = []

    # 1) Login location jump
    dist_login = haversine(*last_loc, *curr_loc)
    if dist_login > 10:
        flags.append("Unusual login distance (>10km)")

    # 2) Travel speed during 30s
    dist_session = haversine(*prev_loc, *latest_loc)
    speed_kmh = dist_session / (30/3600)
    if speed_kmh > 80:
        flags.append(f"Abnormal session speed ({speed_kmh:.1f} km/h)")

    # 3) Behavioral extremes
    if v[0] != 0 and v[0] < 50:
        flags.append("Suspicious accuracy")
    if v[1] != 0 and v[1] > 600:
        flags.append("Flight time too high")
    if v[5] != 0 and v[5] > 10:
        flags.append("Error rate too high")

    # 4) Low activity
    if np.count_nonzero(v) < 6:
        flags.append("Low activity this interval")

    return flags, speed_kmh

def update_reference(D_ref, D_std, v, alpha_mean, alpha_std):
    # EMA update
    new_D_ref = alpha_mean * v + (1 - alpha_mean) * D_ref
    new_D_std = alpha_std * np.abs(v - new_D_ref) + (1 - alpha_std) * D_std
    return new_D_ref, new_D_std

def safe_to_update(v, D, D_std):
    if np.count_nonzero(v) < 6:
        return False
    z = np.abs((v - D) / (D_std + 1e-8))
    score = np.mean(z)
    return score < distance_update_threshold


# Compute std
D_std = estimate_std(D_ref)

# Anomaly score and flags
an_score, z_scores = compute_anomaly_score(v, D_ref, D_std, age)
flags, speed = rule_based_checks(v, last_login_loc, current_loc, prev_loc, latest_loc)

# Thresholds
T_PASS = 1.5
T_ESC_T2 = 2.5

# Decision logic
if an_score is None:
    idle_count += 1
    if idle_count >= idle_threshold and speed > 80:
        decision = "ESCALATE TO T2 (idle + fast travel)"
    else:
        decision = "SKIP INTERVAL (idle)"
else:
    idle_count = 0
    if an_score < T_PASS and not flags:
        decision = "PASS"
    elif an_score < T_ESC_T2 or flags:
        decision = "ESCALATE TO T2"
    else:
        decision = "ESCALATE TO T3"

# Update reference (only if PASS and valid)
if decision == "PASS" and safe_to_update(v, D_ref, D_std):
    D_ref, D_std = update_reference(D_ref, D_std, v, alpha_mean, alpha_std)
    print(f"✔️ Reference profile updated.\n New D_ref:{D_ref}\n New D_std:{D_std}")

# Output
print(f"\nv:           {v}")
print(f"z-scores:    {z_scores.round(2) if z_scores is not None else 'N/A'}")
print(f"anomaly:     {an_score:.2f}" if an_score else "anomaly: N/A")
print(f"flags:       {flags}")
print(f"decision:    {decision}")